# ImcPQ

### @Version3 Nov 2022 by Somi Afiuni

### This module provides IMC segmentaion and expressions.

In [2]:
import os
from os import listdir
from glob import glob
import pathlib
import re
import pandas as pd
import numpy as np
from numpy import array
from imctools.converters import ome2analysis
from imctools.converters import ome2histocat
from imctools.converters import mcdfolder2imcfolder
from imctools.converters import exportacquisitioncsv
import skimage
from skimage.transform import rescale, resize
from skimage import exposure
from shutil import copyfile
from skimage.io import imread, imsave
from matplotlib import pyplot as plt
from deepcell.utils.plot_utils import create_rgb_image
from deepcell.applications import Mesmer
from imcpq_exp import get_panel, df_measured

In [5]:
folder_path_base = './analysis'
folder_data_base = './data'
folder_config_base = './config'
folders_path_inputs = ['./data']
input_file_regexp = '.*.zip'
file_path_csv_panel = './config/Panel.csv'
csv_panel_metal = 'MetalTag'
csv_panel_full = 'full'
pathlib.Path(folder_path_base).mkdir(parents=True, exist_ok=True)
pathlib.Path(folder_data_base).mkdir(parents=True, exist_ok=True)
pathlib.Path(folder_config_base).mkdir(parents=True, exist_ok=True)
folder_path_base = pathlib.Path(folder_path_base)
folders_path_inputs = [pathlib.Path(f) for f in folders_path_inputs]
folder_path_analysis = folder_path_base / 'tiffs'
folder_path_ome= folder_path_base / 'ometiff'
folder_path_cp = folder_path_base / 'output'
folder_path_cp_input = folder_path_base / 'output'
suffix_full = '_full'
suffix_mask = '_mask.tiff'
failed_images = list()
for fol in [folder_path_analysis,
            folder_path_ome, folder_path_cp]:
    if not fol.exists():
        fol.mkdir(parents=True)

### Save zipped data (including an mcd file) in ImcPQ > data folder
### Save panel as "Panel" in the ImcPQ> config folder. Panel should include MetalTag, Target, full, and IA columns.

In [17]:
failed_images = list()
re_fn = re.compile(input_file_regexp)

for fol in folders_path_inputs:
    for fn in fol.glob('*'):
        if re_fn.match(fn.name):
            mcdfolder2imcfolder.mcdfolder_to_imcfolder(fn, output_folder=folder_path_ome)
exportacquisitioncsv.export_acquisition_csv(folder_path_ome, output_folder=folder_path_cp_input)
list_analysis_stacks =[
    (csv_panel_full, suffix_full, 0)]
            

In [18]:
ome2analysis.omefolder_to_analysisfolder(folder_path_ome, folder_path_analysis, panel_csv_file=file_path_csv_panel,
                                         analysis_stacks=(list_analysis_stacks), metalcolumn=csv_panel_metal)
print(ome2analysis.omefolder_to_analysisfolder)

<function omefolder_to_analysisfolder at 0x7fc9b1cbd550>


## Sample Map

In [19]:
csv_ac_md = os.path.join(folder_path_cp_input, 'acquisition_metadata.csv')
ac_md = pd.read_csv(csv_ac_md, sep=',')
ac_md['TiffName'] = ac_md.AcSession.map(str)+"_s0"+"_a"+ ac_md.id.map(str)+"_ac_full.tiff"
ac_md['ROI_id'] = ac_md.AcSession.map(str) +'_'+ ac_md.id.map(str)
tiffToSample = ac_md[['TiffName', 'description', 'ROI_id']]
tiffToSample = tiffToSample.rename(columns = {'description' : 'Sample', 'ROI_id' : 'SampleName'})
tiffToSample.to_csv('./analysis/output/Sample.csv', index=False)

## IA Generating

In [22]:
folder_path_base = pathlib.Path(folder_path_base)
folder_path_analysis = folder_path_base / 'tiffs'
folder_path_base = './analysis'
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]
csv_tiff_files = find_csv_filenames(folder_path_analysis)
csv_channel_seq = os.path.join(folder_path_analysis, csv_tiff_files[0])
df_ch = pd.read_csv(csv_channel_seq, sep=',', header=None)
df_ch.columns = ['MetalTag']
df = pd.read_csv(file_path_csv_panel, sep=',')
df = df[df['full'] == 1]
join_df = df_ch.set_index('MetalTag').join(df.set_index('MetalTag'))
join_df['metal_ch_index']= range(1, len(join_df) + 1)
IA = list(set(join_df[join_df.IA>0].IA)) 
start_dir = folder_path_analysis



In [25]:
for file in os.listdir(start_dir):
    if '_full.tiff' in file:
        full_tiff_file = os.path.join(start_dir, file)
        IA_file_name = file.replace("_full.tiff", "_IA.tiff")
        im = imread(full_tiff_file)
        c, h, w = im.shape
        if h < c: #the index error happens when number of channels in included in full.tiff is less than number of pixels. 50 is good rule of thumb to avoid this error
            print(h, "height less", IA_file_name)
            continue
        if w < c:  #the index error happens when number of channels in included in full.tiff is less than number of pixels. 50 is good rule of thumb to avoid this error
            print(w, "width too low", IA_file_name)
            continue
        if im.shape[0]<im.shape[1] and im.shape[0]<im.shape[2] : 
            im = np.rollaxis(im, 0, 3) 
            IA_stack = np.zeros((im.shape[0],im.shape[1],3))
            for cats in IA: 
                j = int(cats) 
                pnl = join_df.loc[(join_df['full'] == 1 ) & (join_df['IA'] == j)]
                ims = im[:, :, pnl['metal_ch_index']-1]
                IA_stack[:,:,j-1] = ims.sum(axis=2)
                IA_stack2 = rescale(IA_stack,1, multichannel=True, anti_aliasing=False, mode='reflect')
                IA_stack2 = np.array(IA_stack2,dtype='uint16')
            fn = os.path.join(os.path.abspath(start_dir), IA_file_name)
            imsave(fn, IA_stack2, check_contrast=False)
        

21 h TMA_2022_PDAC8_s0_a134_ac_IA.tiff
21 h TMA_2022_PDAC8_s0_a133_ac_IA.tiff


## Segmentation

### Add Padding to Smallest Images

In [ ]:
#add padding to the smaller images (the ones less than 256), to prevent them from failing segmentation
#all the ones that fail are less than 256 x 256, although not all images less than 256x256 fail
base_path = './analysis/tiffs'
for file in os.listdir(base_path):
    if 'IA.tiff' in file:
        iao = os.path.join(base_path, file)
        im = imread(iao)
        im = im.transpose(2,0,1)  #transpose to work with padding
        c, h, w,  = im.shape
        #print(len(im))
        if h < 256:
            print("Oh No - height too low")
            im = np.asarray([np.pad(x, pad_width=((256 - h, 0), (0,0) ), mode='constant', constant_values= 0,) for x in im])
            #print(im.shape)
        #print(w)
        if w < 256:
            print("Oh No - width too low") 
            im = np.asarray([np.pad(x, pad_width=((0, 0), (256-w,0) ), mode='constant', constant_values= 0,) for x in im])
            #print(im.shape)
        im = im.transpose(1,2,0)   #transpose back after padding
        #print(im.shape)
        for_tiff = np.array(im, dtype='uint16') 
        fn = os.path.join(base_path, file)  
        imsave(fn, for_tiff, check_contrast=False)

### Deepcell Segmentation

In [ ]:
base_path = './analysis/tiffs'
for file in os.listdir(base_path):
    if 'IA.tiff' in file:
        ex_mask_fn = file.replace('IA.tiff', 'IA_mask.tiff')
        if os.path.isfile(os.path.join(base_path, ex_mask_fn)):
            continue
        iao = os.path.join(base_path, file)
        im = imread(iao)
        image = np.expand_dims(im, axis=0)
        rgb_im = create_rgb_image(image[:, :, :, 0:2], channel_colors=['green', 'blue'])
        app = Mesmer()
        seg_pred = app.predict(image[:, :, :, 0:2], image_mpp=0.5)
        for_tiff = np.array(seg_pred[0],dtype='uint16')
        fn = os.path.join(base_path, file.replace('IA.tiff', 'IA_mask.tiff'))
        imsave(fn, for_tiff, check_contrast=False)

### Remove Padding

In [1]:
#remove border from masks
#aka, remove the difference between the full tiff file and the IA file
base_path = './analysis/tiffs'
for file in os.listdir(base_path):
    if 'IA_mask.tiff' in file:
        mask_tiff_name = os.path.join(base_path, file)
        full_tiff_name = file.replace('_IA_mask', '_full')
        IA_tiff_name = file.replace('_IA_mask', '_IA')
        
        mask_im = imread(mask_tiff_name)
        full_im = imread(os.path.join(base_path, full_tiff_name))
        IA_im = imread(os.path.join(base_path, IA_tiff_name))
        
        h_padded, w_padded, c_padded = IA_im.shape
        h, w, c = full_im.shape
        
        height_to_remove = h_padded - h
        width_to_remove = w_padded - w

        if height_to_remove != 0:
            mask_im = mask_im[height_to_remove : h_padded, 0 : w_padded]
        if width_to_remove != 0:
            mask_im = mask_im[0: h , width_to_remove : w_padded]
        
        for_tiff = np.array(mask_im, dtype='uint16') 
        imsave(mask_tiff_name, for_tiff, check_contrast=False)

NameError: name 'os' is not defined

## Expressions

In [9]:
#Reading files
current_path = os.path.abspath(os.getcwd())
folder_path_base = os.path.join(current_path, 'analysis', 'tiffs')
folder_data_base = os.path.join(current_path, 'data')
file_path_csv_panel = os.path.join(current_path, 'config', 'Panel.csv') 
file_path_sample = os.path.join(current_path, 'analysis', 'output', 'Sample.csv')
output_measurments_file = os.path.join(current_path, 'analysis', 'output', 'measurements.csv')
sample_df = pd.read_csv(file_path_sample)
panel_df = pd.read_csv(os.path.join(file_path_csv_panel))
panel = get_panel(folder_path_base, panel_df)

In [11]:
expressions = df_measured(folder_path_base, sample_df, panel)
expressions.to_csv('./analysis/output/measurements.csv', index=False)